## Visualizing the temperature and energy transport in the ocean, where major mixing happens along the global ocean conveyor belt. The vizualization portrays the flow over the entire year of 2019, and spans the reigion with coordinates of [-20, 80] degrees North and [-80, 20] degrees East. 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import xarray as xr

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [ ]:
# Importing the data needed for the visualizations. Data was downloaded from CMEMS website in NetCDF format.

filepath = 'YearlyData/'

# Two datasets were used. One near the surface, and one that at a 110m depth.
# Uncomment line for the data that you want to visualize, as appropriate. Otherwise, use different variable names
# for each set.

data = xr.open_mfdataset(filepath+'*_surface', combine='by_coords')
# data = xr.open_mfdataset(filepath+'*_110m', combine='by_coords')


In [ ]:
# Data contains depth, time, spatial coordinates, eastward and northward velocity, and sea water potential temperature.
# Units are m, sec, degrees N, degrees E, m/s, and degrees Celcius, respectively.

depth = data.depth
time = data.time
lons, lats = data.longitude, data.latitude
u, v = data.uo, data.vo
temp = data.thetao

In [ ]:
# Create numpy arrays of the data.

d = depth.values
lat, lon = lats.values, lons.values
x2, y2 = np.meshgrid(lon, lat)
vx, vy = u.values, v.values
T = temp.values
t = time.values

# Data has shape of (time, depth, latitude, longitude). 

T.shape

In [ ]:
# Define special colormap for Energy transport visualization for better clarity.

color = ['white', 'red','red']
nodes = [0.0, 0.1, 1.0]
mycmap = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, color)))

In [ ]:
# Define figure size and subplot properties and axes projection to be used for plotting the map. 

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5,8) ,sharex='all', sharey='all',
                            subplot_kw={'projection': ccrs.PlateCarree()})

In [ ]:
# Defining the animation loop function, which contains main plotting commands.

def update(frame):

    fig, (ax1, ax2) = plt.subplots(2, 1, num=1, figsize=(5,8) ,sharex='all', sharey='all',
                                subplot_kw={'projection': ccrs.PlateCarree()}, clear=True)
    plt.subplots_adjust(hspace=0.05)

    # Add the coastlines on map.
    ax1.add_feature(cfeature.LAND, edgecolor='black',facecolor='black')
    ax2.add_feature(cfeature.LAND, edgecolor='black',facecolor='black')

    # Temperature
    T0 = T[frame]
    T0 = T0.reshape((lats.shape[0], lons.shape[0]))
    
    # Velocity
    vx0, vy0 = vx[frame], vy[frame]
    vx0 = vx0.reshape((lats.shape[0], lons.shape[0]))
    vy0 = vy0.reshape((lats.shape[0], lons.shape[0]))

    # Calculate Kinetic Energy per unit mass, which we will use to visualize energy transport in the ocean.
    KE = 0.5*(vx0**2 + vy0**2)

    # Plot the temperature data.
    Tcm = ax1.pcolormesh(lon, lat, T0, cmap='jet', vmin=-3, vmax=32, transform=ccrs.PlateCarree())
    Tcbar = fig.colorbar(Tcm, ax=ax1, orientation='vertical', label='T ($^\circ$C)')

    # Plot the KE data.
    Ecm = ax2.pcolormesh(lon, lat, KE, cmap=mycmap, vmin=0, vmax=5, transform=ccrs.PlateCarree())
    Ecbar = fig.colorbar(Ecm, ax=ax2, ticks=[0, 5], orientation='vertical', label='KE per unit mass ($m^2/s^2$)')

    ax1.set_title(str(t[frame])[0:10])

    ax1.set_xlim([lon.min(), lon.max()])
    ax1.set_ylim([lat.min(), lat.max()])

    ax2.set_xticks(range(int(lon.min()), int(lon.max()), 15))
    ax1.set_yticks(range(int(lat.min()), int(lat.max()), 15))
    ax2.set_yticks(range(int(lat.min()), int(lat.max()), 15))

plt.show()


In [ ]:
# Execute the animation loop function; save the animation as mp4 at 15fps and resolution of 300dpi 

anim = FuncAnimation(fig, update, frames=np.arange(0,len(T)), repeat=False)
anim.save('2019_surface.mp4', writer='ffmpeg', dpi=300, fps=15)

In [50]:
# This is how the output should eventually look like.

from IPython.display import HTML, display, display_markdown

display_markdown('## $\hspace{50pt}$ Depth = 0.5 m $\hspace{120pt}$ Depth = 110 m', raw=True)
display(HTML("<table><tr><td><video controls src=2019_surface.mp4 width=500 allowfullscreen></td><td><video controls src=2019_110m.mp4 width=500 allowfullscreen></td></tr></table>"))



## $\hspace{50pt}$ Depth = 0.5 m $\hspace{120pt}$ Depth = 110 m

,


In [ ]:
# from IPython.display import HTML, display, display_markdown
# from base64 import b64encode

# video_surface = open("2019_surface.mp4", "rb").read()
# video_110m = open("2019_110m.mp4", "rb").read()

# video_encoded1 = b64encode(video_surface).decode('ascii')
# video_encoded2 = b64encode(video_110m).decode('ascii')

# video_tag1 = '<video controls alt="test" src="data:video/x-m4v;base64,{0}" width="600" height="600 allowfullscreen">'.format(video_encoded1)
# video_tag2 = '<video controls alt="test" src="data:video/x-m4v;base64,{0}" width="600" height="600 allowfullscreen">'.format(video_encoded2)

# display_markdown('## Depth = 0.5 m', raw=True)
# display(HTML(data=video_tag1))

# # display_markdown('## Depth = 110 m', raw=True)
# # display(HTML(data=video_tag2))

In [1]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="2019_surface.mp4" type="video/mp4">
</video></div>